In [ ]:
# Setup the Jupyter version of Dash
from dash import Dash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
import math

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import for CRUD module
from crud import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# Username and password and CRUD Python module name

# username = "aacuser"
# password = "Mercuryaac@1254"

# Connect to database via CRUD Module
db = AnimalShelter()

# build frequently used indexes
db.frequent_indexes()

# indexes utilized in UI
PROJECTION = {
    "_id": 0,
    "animal_id": 1,
    "name": 1,
    "animal_type": 1,
    "breed": 1,
    "sex_upon_outcome": 1,
    "age_upon_outcome_in_weeks": 1,
    "outcome_type": 1,
    "location_lat": 1,
    "location_lon": 1,
}

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
# df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
# df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = Dash('ProjectTwoDashboard')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
#Add in code for the interactive filtering radio button options.
app.layout = html.Div([
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '100px'}),
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('CS-340 Dashboard (Chase Barnes)'))),
    html.Hr(),
    html.Label('Select Rescue Type:'),
    dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
            {'label': 'Reset', 'value': 'Reset'},
        ],
        value='Reset',
        inline=True
    ),
    html.Hr(),
#Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in PROJECTION if PROJECTION[i] != 0], 
                         data=[], filter_action="none", sort_action="none", sort_mode="multi",
                         column_selectable=False, row_selectable=False, row_deletable=False, selected_columns=[],
                         selected_rows=[], page_action="custom", page_current=0, page_size=10, page_count=0, style_cell={ 
                             'textAlign': 'left', 'whiteSpace': 'normal'}
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback([Output('datatable-id','data'), Output('datatable-id', 'page_count')],
              [Input('filter-type', 'value'), Input('datatable-id', 'page_current'), Input('datatable-id', 'page_size')])
def update_dashboard(filter_type, page_current, page_size):
    # Retrieve select filter using Crud match_rescue    
    if filter_type == 'Reset':
        query = {}
    else:
        try:
            query = db.match_rescue(filter_type)
        except ValueError:
            query = {}

    total = db.count(query)
    page_count = max(1, math.ceil(total / max(1, int(page_size))))

    results = db.read_page(query=query, projection=PROJECTION, sort=[("age_upon_outcome_in_weeks", 1)], 
                           page =int(page_current) + 1, page_size=int(page_size))

    return pd.DataFrame.from_records(results).to_dict('records'), page_count

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('filter-type', "value")])
def update_graphs(filter_type):
    # Use aggregation pipeline on Reset
    # Use cached rescue aggregation otherwise
    if filter_type == 'Reset':
        result = db.aggregate([
            {"$group": {"_id": "$outcome_type", "count": {"$sum": 1}}},
            {"$sort": {"count": -1}}
        ], limit=1000)
    else:
        result = db.cached_count_by_outcome_rescue(filter_type, ttl_s=300)

    dff = pd.DataFrame(result).rename(columns={"_id": "outcome_type"})
    if dff.empty:
        return []
    return [
        dcc.Graph(
            figure=px.pie(
                dff,
                names="outcome_type",
                values="count",
                title=f"Outcome distribution — {filter_type if filter_type!='Reset' else 'All Animals'}"
            )
        )
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    default_center = [30.75, -97.48]

    # no rows yet → show base map
    if not viewData:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=default_center, zoom=10,
                   children=[dl.TileLayer(id="base-layer-id")])
        ]

    dff = pd.DataFrame.from_dict(viewData)

    # pick first row if nothing selected / empty list
    if not index:
        row = 0
    else:
        row = index[0] if 0 <= index[0] < len(dff) else 0

    # if coordinates are missing, just show base map
    if 'location_lat' not in dff.columns or 'location_lon' not in dff.columns:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=default_center, zoom=10,
                   children=[dl.TileLayer(id="base-layer-id")])
        ]

    try:
        lat = float(dff.iloc[row]["location_lat"])
        lon = float(dff.iloc[row]["location_lon"])
    except Exception:
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=default_center, zoom=10,
                   children=[dl.TileLayer(id="base-layer-id")])
        ]

    breed = dff.iloc[row].get("breed", "Unknown")
    name = dff.iloc[row].get("name", "Unknown")

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=12, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([html.H1("Animal Name"), html.P(name)])
            ])
        ])
    ]



app.run(debug=False, port=8051,jupyter_mode="external")


c:\Users\chazz\AppData\Local\Programs\Python\Python313\Lib\site-packages\dash\dash.py:634: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



AttributeError: 'super' object has no attribute 'run_server'